In [21]:
import os
import cv2
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import InputLayer

In [6]:
mnist = tf.keras.datasets.mnist

(MNx_train, MNy_train), (MNx_test, MNy_test) = mnist.load_data()

MNx_train = MNx_train.reshape(MNx_train.shape[0], -1)
MNx_test = MNx_test.reshape(MNx_test.shape[0], -1)

Make sure to replace the following path with wherever you place the A_Z Handwritten data from https://www.kaggle.com/datasets/sachinpatel21/az-handwritten-alphabets-in-csv-format/data

In [7]:
data = pd.read_csv('D:/Pycharm Projects/A_Z Handwritten Data/A_Z Handwritten Data.csv')

data.columns = ['label'] + [f'pixel_{i}' for i in range(784)]
x = data.drop('label', axis=1)
y = data['label']

AZx_train, AZx_test, AZy_train, AZy_test = train_test_split(x, y, test_size=0.2)

In [8]:
for x,y in AZy_train.items():
    y += 10
for _,y in AZy_test.items():
    y += 10

In [9]:
MNx_train = tf.keras.utils.normalize(MNx_train, axis=1)
MNx_test = tf.keras.utils.normalize(MNx_test, axis=1)

AZx_train = tf.keras.utils.normalize(AZx_train, axis=1)
AZX_test = tf.keras.utils.normalize(AZx_test, axis=1)

In [10]:
TrainingData = tf.concat([MNx_train, AZx_train], axis=0)
TrainingLabels = tf.concat([MNy_train, AZy_train], axis=0)
TestData = tf.concat([MNx_test, AZx_test], axis=0)
TestLabels = tf.concat([MNy_test, AZy_test], axis=0)

In [25]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.InputLayer(shape=(AZx_train.shape[1],)))
    # model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(36, activation='softmax'))

model.compile(optimizer=Adam(learning_rate = 0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 36)             │         2,340 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 111,076 (433.89 KB)

 Trainable params: 111,076 (433.89 KB)

 Non-trainable params: 0 (0.00 B)

None


In [26]:
model.fit(TrainingData, TrainingLabels, epochs = 10, validation_data=(TestData, TestLabels))

Epoch 1/10
11187/11187 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - accuracy: 0.5335 - loss: 1.7401 - val_accuracy: 0.7644 - val_loss: 822.5042
Epoch 2/10
11187/11187 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - accuracy: 0.8009 - loss: 0.7157 - val_accuracy: 0.8206 - val_loss: 670.5433
Epoch 3/10
11187/11187 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - accuracy: 0.8514 - loss: 0.5413 - val_accuracy: 0.8463 - val_loss: 602.2214
Epoch 4/10
11187/11187 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - accuracy: 0.8781 - loss: 0.4395 - val_accuracy: 0.8694 - val_loss: 504.8229
Epoch 5/10
11187/11187 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - accuracy: 0.8954 - loss: 0.3772 - val_accuracy: 0.8697 - val_loss: 527.0475
Epoch 6/10
11187/11187 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - accuracy: 0.9084 - loss: 0.3321 - val_accuracy: 0.8726 - val_loss: 521.5330
Epoch 7/10
11187/11187 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - accuracy: 0.9173 - loss: 0.3000 - val_accuracy: 0.8787 - val_loss: 505.4102
Epoch 8/10
11187/11187 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/ste

In [27]:
MNloss, MNaccuracy = model.evaluate(MNx_test, MNy_test)
AZloss, AZaccuracy = model.evaluate(AZx_test, AZy_test)

print(f"MNIST Test loss: {MNloss}. \n MNIST Test accuracy: {MNaccuracy*100:.2f}")

print(f"A_Z Test loss: {AZloss}. \n A_Z Test accuracy: {AZaccuracy*100:.2f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 574us/step - accuracy: 0.8792 - loss: 0.3919
2328/2328 ━━━━━━━━━━━━━━━━━━━━ 2s 641us/step - accuracy: 0.8655 - loss: 686.1584
MNIST Test loss: 0.3875495493412018. 
 MNIST Test accuracy: 88.07
A_Z Test loss: 693.829833984375. 
 A_Z Test accuracy: 86.59


In [32]:
image_number = 0
while os.path.isfile(f"D:\Pycharm Projects\digits\digit{image_number}.png"):
    img = cv2.imread(f"D:\Pycharm Projects\digits\digit{image_number}.png")[:, :, 0]
    img = np.invert(np.array([img]))
    img = np.reshape(img, (1,784))
    prediction = model.predict(img)
    print(f"The Image number {image_number} is probably {np.argmax(prediction)}")
    image_number += 1
    

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
The Image number 0 is probably 6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
The Image number 1 is probably 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
The Image number 2 is probably 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
The Image number 3 is probably 18
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
The Image number 4 is probably 4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
The Image number 5 is probably 4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
The Image number 6 is probably 4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
The Image number 7 is probably 19
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
The Image number 8 is probably 19
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
The Image number 9 is probably 17
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
The Image number 10 is probably 24
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
The Image number 11 is probably 19
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
The Image number 12 is probably 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
The Image number 13 is probabl